# 필요한 라이브러리 및 모듈 설치

In [ ]:
!pip install -qU langchain openai chromadb tiktoken pypdf faiss-cpu langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 12.2 MB/s et

# Google Drive 마운트 및 작업 디렉토리 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/miraeasset/code')

# 필요한 라이브러리 및 모듈 import

In [ ]:
import json
import requests
import pickle
from operator import itemgetter

from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS, Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.language_models.llms import LLM
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.documents import Document
from langchain.chains import LLMChain

from custom_llm import CustomLLM
from embedding import HyperClovaEmbedding, HyperClovaEmbeddings
# CustomLLM class 코드는 custom_llm.py 에서 확인 가능.
# HyperClovaEmbedding, HyperClovaEmbeddings class 코드는 embedding.py 에서 확인 가능.

# HyperClovaEmbedding 인스턴스 생성 및 임베딩 테스트

In [ ]:
hyper_clova = HyperClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)
# 개인 HyperClovaEmbedding API를 입력

try:
    text = "input text"
    embedding = hyper_clova.execute(text)
    print(f"Embedding: {embedding}")
except ValueError as e:
    print(e)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

embedding_func = HyperClovaEmbeddings(hyper_clova)

Embedding: [-0.9326172, 0.69628906, -0.515625, 0.12683105, -0.29736328, -0.55615234, 0.2388916, 0.15234375, 0.40039062, -0.68359375, 0.29541016, -0.047424316, 0.36669922, -0.81689453, -0.2619629, -0.34375, -0.16601562, 0.36743164, 0.17993164, -0.2705078, -0.80078125, -0.24499512, -0.08093262, 0.34033203, 0.47387695, 0.9682617, 0.2866211, -0.42407227, -0.58154297, -0.20178223, 0.87646484, 0.0181427, -0.7939453, -2.0625, 0.31201172, -0.11657715, -0.12573242, -0.3149414, -1.5751953, 0.27807617, 0.087524414, -0.59716797, 0.55615234, -0.022842407, -0.27807617, -0.081970215, 1.0087891, -0.39575195, -0.7050781, -0.5683594, 0.054107666, 0.52685547, 1.8037109, -0.1784668, -0.31225586, -0.5307617, -0.2388916, 0.16149902, -1.6572266, -0.07635498, -0.2788086, -0.54296875, -0.48706055, -0.27929688, -0.21374512, 1.7119141, 0.91308594, 0.13464355, -0.703125, -0.67089844, -0.33496094, 0.7866211, -0.3413086, -0.08496094, -1.2548828, 1.1201172, 0.6401367, -0.40893555, -0.41479492, 0.6435547, 0.6928711, 

# Chroma DB 설정 및 MultiVectorRetriever 생성 함수 정의

In [ ]:
chroma_db_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_"

def create_retriever(name):
    db = Chroma(
        collection_name="summaries",
        embedding_function=embedding_func,
        persist_directory=chroma_db_path + name
    )
    with open(f"{chroma_db_path}{name}/db_{name}_docstore.pkl", "rb") as f:
        store = pickle.load(f)
    retriever = MultiVectorRetriever(
        vectorstore=db,
        docstore=store,
        id_key="doc_id"
    )
    return retriever

retriever_기업 = create_retriever("company")
retriever_경제 = create_retriever("economy")
retriever_세법 = create_retriever("law")
retriever_미래에셋 = create_retriever("miraeasset")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


# CustomLLM 인스턴스 생성

In [ ]:
llm = CustomLLM(
    host='https://clovastudio.stream.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)

# 사용자 정보 입력 및 PromptTemplate 설정

In [ ]:
def create_prompt(user_info):
    template = """
    아래는 사용자와 AI의 대화 예시야. 잘 보고 대답해줘.

    사용자 : 저축을 얼마나 해야해
    AI :저축은 개인의 상황에 따라 다르기 때문에 일반적인 기준은 없지만, 보통 소득의 20% 이상을 저축하는 것이 권장됩니다. 하지만 이는 개인의 목표와 상황에 따라 다를 수 있습니다.

    현재 당신의 소득은 연 근로소득 11200만원, 불로소득 300만원 (주식 배당금)으로 총 11500만원입니다. 이를 바탕으로 계산하면, 연 저축 목표는 11500만원 x 20% = 2300만원 이상이 되는 것이 좋습니다.

    저축 목표를 세울 때는 자신의 상황과 목표를 고려하여 적절한 금액을 설정하는 것이 중요합니다. 저축을 통해 미래에 필요한 자금을 마련하고, 경제적 안정을 추구할 수 있습니다. 💰

    사용자 : 저축을 얼마나 해야할까?
    AI :저축은 개인의 상황에 따라 다르기 때문에 일반적인 기준은 없지만, 보통 소득의 20% 이상을 저축하는 것이 권장됩니다. 하지만 이는 개인의 목표와 상황에 따라 다를 수 있습니다.

    현재 당신의 소득은 연 근로소득 11200만원, 불로소득 300만원 (주식 배당금)으로 총 11500만원입니다. 이를 바탕으로 계산하면, 연 저축 목표는 11500만원 x 20% = 2300만원 이상이 되는 것이 좋습니다.

    저축 목표를 세울 때는 자신의 상황과 목표를 고려하여 적절한 금액을 설정하는 것이 중요합니다. 저축을 통해 미래에 필요한 자금을 마련하고, 경제적 안정을 추구할 수 있습니다. 💰

    사용자 : 저축을 얼마나 해야하는지 말해줄래?
    AI :저축은 개인의 상황에 따라 다르기 때문에 일반적인 기준은 없지만, 보통 소득의 20% 이상을 저축하는 것이 권장됩니다. 하지만 이는 개인의 목표와 상황에 따라 다를 수 있습니다.

    현재 당신의 소득은 연 근로소득 11200만원, 불로소득 300만원 (주식 배당금)으로 총 11500만원입니다. 이를 바탕으로 계산하면, 연 저축 목표는 11500만원 x 20% = 2300만원 이상이 되는 것이 좋습니다.

    저축 목표를 세울 때는 자신의 상황과 목표를 고려하여 적절한 금액을 설정하는 것이 중요합니다. 저축을 통해 미래에 필요한 자금을 마련하고, 경제적 안정을 추구할 수 있습니다. 💰

    사용자 : 저축을 얼마나 해야할까
    AI :저축은 개인의 상황에 따라 다르기 때문에 일반적인 기준은 없지만, 보통 소득의 20% 이상을 저축하는 것이 권장됩니다. 하지만 이는 개인의 목표와 상황에 따라 다를 수 있습니다.

    현재 당신의 소득은 연 근로소득 11200만원, 불로소득 300만원 (주식 배당금)으로 총 11500만원입니다. 이를 바탕으로 계산하면, 연 저축 목표는 11500만원 x 20% = 2300만원 이상이 되는 것이 좋습니다.

    저축 목표를 세울 때는 자신의 상황과 목표를 고려하여 적절한 금액을 설정하는 것이 중요합니다. 저축을 통해 미래에 필요한 자금을 마련하고, 경제적 안정을 추구할 수 있습니다. 💰

    #########
    당신은 미래에셋증권에서 서비스되는 개인 금융비서입니다.

    내 정보는 다음과 같습니다.

    나이: {age}세
    성별: {gender}
    직업: {job} (근무 경력 {job_experience}년)
    가족 구성: {family_members}명 (자녀 {children_count}명 포함)

    소득
    연 근로소득: {annual_income}만원
    연 불로소득: {passive_income}만원 (주식 배당금)

    자산
    총 자산: {total_assets}만원
    자산 구성: {assets}

    일반적인 답변보다 최대한 내 정보에 맞는 구체적인 답변을 하십시오.
    답을 모르면 그냥 모른다고 대답하십시오.
    금융과 상관없는 질문도 성의껏 대답하십시오.
    무조건 답변을 하고 절대 빈칸으로 답변하지 마십시오.
    다음과 같은 #맥락과 #채팅히스토리를 참고하여 마지막 질문에 대답하십시오.

    #맥락: {context}
    #채팅히스토리: {chat_history}
    질문: {question}
    도움이 되는 답변:
    """
    prompt = template.format(
        age=user_info['age'],
        gender=user_info['gender'],
        job=user_info['job'],
        job_experience=user_info['job_experience'],
        family_members=user_info['family_members'],
        children_count=user_info['children_count'],
        annual_income=user_info['annual_income'],
        passive_income=user_info['passive_income'],
        total_assets=user_info['total_assets'],
        assets=user_info['assets'],
        context='{context}',
        chat_history='{chat_history}',
        question='{question}'
    )
    return prompt

In [ ]:
user_info = get_user_input()

In [ ]:
# user_info 예시
# user_info = {
#     'age': 48,
#     'gender': '남성',
#     'job': '회사원 (중견기업)',
#     'job_experience': 22,
#     'family_members': 4,
#     'children_count': 2,
#     'annual_income': 11200,
#     'passive_income': 300,
#     'total_assets': 103000,
#     'assets': '주택 (아파트): 83000 만원 (주택담보대출: 21000 만원), 현금: 10000만원 , 미국 S&P ETF: 10000만원'
# }

# 카테고리 분류기 클래스 정의 및 세션 기록 저장 딕셔너리

In [ ]:
class CategoryClassifier:
    def __init__(self, retriever_기업, retriever_경제, retriever_세법, retriever_미래에셋):
        self.retriever_기업 = retriever_기업
        self.retriever_경제 = retriever_경제
        self.retriever_세법 = retriever_세법
        self.retriever_미래에셋 = retriever_미래에셋

        fewshot_prompt = """
        아래는 사용자와 AI의 대화 예시야. 잘 보고 대답해줘.
        예시와 같이 카테고리만 반환해.
        다른 말은 하지마.
        어느 카테고리에도 속하지 않으면 '기타' 를 반환해.

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 내가 주식으로 5000만원 벌었는데 세금이 얼마일까?
        AI : 세법

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 네이버의 주가 전망 말해줘
        AI : 기업

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : SK하이닉스의 재무재표 알려줘
        AI : 기업

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 네이버의 작년 2분기 매출 알려줘
        AI : 기업

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : sk하이닉스와 유사한 기업에 대해 말해봐
        AI : 기업

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 미래에셋의 ESG경영에 대해 알려줘
        AI : 미래에셋

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 미래에셋의 중장기 전략이 뭐야?
        AI : 미래에셋

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 미래에셋의 내부감사 실시 현황 알아?
        AI : 미래에셋

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 내 포트폴리오 분석해줘
        AI : 경제

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 월급의 얼마를 저축해야 할까?
        AI : 경제

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 투자상품 추천해줘
        AI : 경제

        사용자 :
        `기업` or `경제` or `세법` or '미래에셋`으로 질문의 카테고리를 분류해줘
        질문 : 노후대비에 대해 조언 부탁
        AI : 경제

        질문 : {query}
        """

        self.prompt_template = PromptTemplate(template=fewshot_prompt, input_variables=["query"])
        self.llm = CustomLLM(
                    host='https://clovastudio.stream.ntruss.com',
                    api_key='your_api_key_here',
                    api_key_primary_val='your_primary_api_key_here',
                    request_id='your_request_id_here'
                  )
        self.llm_chain = LLMChain(prompt=self.prompt_template, llm=self.llm)

    def select_retriever(self, category):
        if category == "기업":
            return self.retriever_기업
        elif category == "경제":
            return self.retriever_경제
        elif category == "세법":
            return self.retriever_세법
        elif category == "미래에셋":
            return self.retriever_미래에셋
        else:
            return self.retriever_경제

    def ask_question(self, query):
        response = self.llm_chain.run({"query": query})
        # 응답에서 'AI :' 이후의 텍스트만 추출
        category = response.split('AI :')[-1].strip()
        return category

# 세션 기록을 저장할 딕셔너리

In [ ]:
store = {}

# Chat 함수 정의

In [ ]:
classifier = CategoryClassifier(retriever_기업, retriever_경제, retriever_세법, retriever_미래에셋)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
# Chat 함수 정의
def chat(query, session_id, user_info, classifier, store):
    classifier = classifier

    # 질문을 분류하여 카테고리 선택
    category = classifier.ask_question(query)

    # 적절한 리트리버 선택
    retriever = classifier.select_retriever(category)

    # RAG 체인 설정
    dynamic_prompt = create_prompt(user_info)
    rag_prompt_custom = PromptTemplate.from_template(dynamic_prompt)
    rag_chain = {"context": itemgetter("question") | retriever, "question": itemgetter("question"), "chat_history": itemgetter("chat_history")} | rag_prompt_custom | llm

    # 세션 기록을 저장할 딕셔너리
    store = store

    # 세션 ID를 기반으로 세션 기록을 가져오는 함수
    def get_session_history(session_ids):
        if session_ids not in store:  # 세션 ID가 store에 없는 경우
            # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
            store[session_ids] = ChatMessageHistory()
        return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

    # 대화를 기록하는 RAG 체인 생성
    rag_with_history = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
        history_messages_key="chat_history",  # 기록 메시지의 키
    )

    # 질문 답변
    response = rag_with_history.invoke(
        # 질문 입력
        {"question": query},
        # 세션 ID 기준으로 대화를 기록합니다.
        config={"configurable": {"session_id": session_id}},
    )

    return category, response

# 메인 함수 정의 및 실행

In [ ]:
session_id = input("세션 ID를 입력하세요: ")

세션 ID를 입력하세요: 미래


In [ ]:
# 메인 함수
if __name__ == "__main__":
    query = input("질문을 입력하세요: ")
    category, answer = chat(query, session_id, user_info, classifier, store)
    print(f"카테고리: {category}")
    print(f"답변: {answer}")

질문을 입력하세요: 저녁메뉴 추천해줘


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


카테고리: 기타
답변: 저녁 메뉴 추천을 위해 몇 가지 팁을 드리겠습니다.

1.**건강한 식사 선택**: 채소, 과일, 단백질 등 영양소가 풍부한 식사를 선택하여 건강을 유지하세요. 샐러드, 그릴 치킨, 생선 요리, 채소 볶음 등을 고려해 볼 수 있습니다.

2.**다양한 맛 즐기기**: 다양한 나라의 요리를 시도해 보세요. 이탈리아 파스타, 일본 초밥, 태국 카레, 멕시코 타코 등 다양한 음식을 즐길 수 있습니다.

3.**간단한 요리 선택**: 바쁜 저녁에는 간단한 요리를 선택하여 시간을 절약할 수 있습니다. 샌드위치, 파스타, 볶음밥, 덮밥 등이 좋은 선택입니다.

4.**가족 선호도 고려**: 가족 구성원의 선호도를 고려하여 메뉴를 선택하세요. 아이들이 좋아하는 음식이나 특별한 요리를 준비할 수도 있습니다.

5.**계절 식재료 활용**: 계절에 맞는 식재료를 활용하여 신선하고 맛있는 요리를 만들어 보세요. 봄에는 나물, 여름에는 수박, 가을에는 사과, 겨울에는 귤 등을 활용할 수 있습니다.

위의 팁을 참고하여 맛있는 저녁 메뉴를 선택해 보세요. 건강하고 즐거운 식사를 즐기세요! 🍜


In [ ]:
# 메인 함수
if __name__ == "__main__":
    query = input("질문을 입력하세요: ")
    category, answer = chat(query, session_id, user_info, classifier, store)
    print(f"카테고리: {category}")
    print(f"답변: {answer}")

질문을 입력하세요: 내 투자 전략에 대해 조언해줘
카테고리: 경제
답변: 투자 전략을 수립할 때는 자신의 투자 목적과 투자 성향을 고려하여 적절한 금융상품을 선택하는 것이 중요합니다.

먼저, 투자 목적을 설정해야 합니다. 투자 목적은 장기적인 목표와 단기적인 목표로 구분할 수 있습니다. 장기적인 목표는 노후 대비나 자녀 교육비 마련 등이 될 수 있고, 단기적인 목표는 결혼 자금 마련이나 주택 구입 자금 마련 등이 될 수 있습니다.

투자 성향은 개인의 성격이나 태도에 따라 다릅니다. 안정형은 수익보다는 안정성을 중시하며, 원금 손실을 최소화하고자 합니다. 반면에 공격형은 수익을 중시하며, 원금 손실을 감수하더라도 높은 수익을 추구합니다.

현재 당신의 자산 구성을 보면, 부동산에 집중되어 있고 금융 자산의 비중이 상대적으로 낮습니다. 이런 경우에는 금융 자산의 비중을 높이는 것이 좋습니다. 이를 위해 주식이나 펀드 등 다양한 금융상품에 분산 투자하는 것을 추천합니다.  

주식 투자를 하실 때는 기업의 실적과 성장성, 그리고 시장 동향을 함께 고려하시는 것이 좋습니다. 그리고 자신의 투자 목적과 성향에 맞는 적절한 포트폴리오를 구성하여 분산 투자하는 것이 중요합니다.  

금융시장에는 워낙 다양한 금융상품이 있기 때문에 구체적인 상품의 선택이나 투자금액, 투자 기간 등은 전문가의 도움을 받아 결정하는 것이 좋습니다. 은행, 증권사 등 금융회사별로 자사의 웹사이트에서 고객의 성향 및 상황에 맞는 포트폴리오 설계 서비스를 제공하는 경우가 많기 때문에 이를 참고해 결정하는 것도 좋은 방법입니다.
